In [1]:
import sys
sys.path.append("../src")  

In [2]:
import torch
import torch.nn.functional as F
import json
import pickle
import math
from modelo_autoregresivo import ModeloAutoregresivo

In [3]:
# Cargar vocabulario y datos
with open("vocab.json", "r", encoding="utf-8") as f:
    vocab = json.load(f)
inv_vocab = {v: k for k, v in vocab.items()}

with open("secuencias.pkl", "rb") as f:
    secuencias = pickle.load(f)

In [10]:
# Crear modelo
vocab_size = len(vocab)
modelo = ModeloAutoregresivo(vocab_size)
modelo.eval()

ModeloAutoregresivo(
  (embedding): Embedding(23, 128)
  (pos_encoder): PositionalEncoding()
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0

In [16]:
# Elegir una secuencia corta para prueba
entrada = secuencias[0][:5]  
entrada_tensor = torch.tensor(entrada).unsqueeze(0) 

In [17]:
# Generación greedy
output = entrada_tensor.clone()
num_tokens_a_generar = 10

for _ in range(num_tokens_a_generar):
    seq_len = output.size(1)
    mask = modelo.generate_causal_mask(seq_len).to(output.device)

    with torch.no_grad():
        emb = modelo.embedding(output) * math.sqrt(modelo.d_model)
        emb = modelo.pos_encoder(emb)
        emb = emb.transpose(0, 1)  # (seq_len, batch, dim)

        logits = modelo.transformer_decoder(emb, emb, tgt_mask=mask)
        logits = modelo.fc_out(logits)

    ultimo_logit = logits[-1, 0, :]  # último token de salida
    siguiente_token = torch.argmax(F.softmax(ultimo_logit, dim=-1)).view(1, 1)
    output = torch.cat([output, siguiente_token], dim=1)

print("Secuencia final:", "".join([inv_vocab[idx.item()] for idx in output[0]]))

Secuencia final: hola umpgápuuuu


In [23]:
# Generar texto a partir de cada secuencia del corpus
for i, entrada in enumerate(secuencias):
    entrada_tensor = torch.tensor(entrada[:5]).unsqueeze(0)  # (1, 5)
    output = entrada_tensor.clone()

    for _ in range(10):  # generar 10 tokens nuevos
        mask = modelo.generate_causal_mask(output.size(1)).to(output.device)

        with torch.no_grad():
            emb = modelo.embedding(output) * math.sqrt(modelo.d_model)
            emb = modelo.pos_encoder(emb).transpose(0, 1)
            logits = modelo.transformer_decoder(emb, emb, tgt_mask=mask)
            logits = modelo.fc_out(logits)

        ultimo_logit = logits[-1, 0, :]
        siguiente_token = torch.argmax(F.softmax(ultimo_logit, dim=-1), dim=-1)
        siguiente_token = siguiente_token.unsqueeze(0).unsqueeze(0)  # (1, 1)
        output = torch.cat([output, siguiente_token], dim=1)

    texto_generado = "".join([inv_vocab[idx.item()] for idx in output[0]])
    print(f"Entrada {i+1}: {texto_generado}")

Entrada 1: hola umpgápuuuu
Entrada 2: cómo cmámuámmmm
Entrada 3: el mompaeaeaeae
Entrada 4: los tkggggggggm
Entrada 5: la mááááááááámk


In [24]:
from generacion import GeneradorTexto

In [26]:
# Crear el generador
generador = GeneradorTexto(modelo, inv_vocab)

# Generar texto a partir de la entrada_tensor
texto_generado = generador.generar_greedy(entrada_tensor, num_tokens=10)

print("Texto generado:", texto_generado)

Texto generado: la mááááááááámk


In [27]:
for i, secuencia in enumerate(secuencias[:5]):  # Cambia el 5 por el número de pruebas que quieras
    entrada = torch.tensor(secuencia[:5]).unsqueeze(0)  # solo los primeros tokens
    texto_generado = generador.generar_greedy(entrada, num_tokens=10)
    texto_original = "".join([inv_vocab[idx] for idx in secuencia])
    print(f"Entrada {i+1}: {texto_original}")
    print(f"Generado:   {texto_generado}")
    print()


Entrada 1: hola mundo
Generado:   hola umpgápuuuu

Entrada 2: cómo estás
Generado:   cómo cmámuámmmm

Entrada 3: el modelo autoregresivo predice tokens uno por uno
Generado:   el mompaeaeaeae

Entrada 4: los transformers usan atención
Generado:   los tkggggggggm

Entrada 5: la máscara causal evita ver el futuro
Generado:   la mááááááááámk

